# **Chapter 8: Data Wrangling: Join, Combine, and Reshape**

## **8.1 Hierarchical Indexing**

In [3]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9), 
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                                [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [4]:
data

a  1    0.302457
   2    0.734773
   3    0.732804
b  1    0.052856
   3   -1.864805
c  1    0.087993
   2    0.354559
d  2    0.685219
   3   -0.084851
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
data["b"]

1    0.052856
3   -1.864805
dtype: float64

In [7]:
data["b" : "c"]

b  1    0.052856
   3   -1.864805
c  1    0.087993
   2    0.354559
dtype: float64

In [8]:
data.loc[["b", "d"]]

b  1    0.052856
   3   -1.864805
d  2    0.685219
   3   -0.084851
dtype: float64

In [10]:
data.loc[:, 2]

a    0.734773
c    0.354559
d    0.685219
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,0.302457,0.734773,0.732804
b,0.052856,NaN,-1.864805
c,0.087993,0.354559,NaN
d,NaN,0.685219,-0.084851


In [12]:
data.unstack().stack()

a  1    0.302457
   2    0.734773
   3    0.732804
b  1    0.052856
   3   -1.864805
c  1    0.087993
   2    0.354559
d  2    0.685219
   3   -0.084851
dtype: float64

In [14]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                   index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']])

In [15]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
frame.index.names = ['key1', 'key2']

In [17]:
frame.columns.names = ["state", "color"]

In [18]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [20]:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
 names=['state', 'color'])

NameError: name 'MultiIndex' is not defined